In [1]:
import secrets

In [2]:
#Function to apply extended euclidien Algorithmn 
def eucalg(a, b):
	a=int(a)
	b=int(b)
	# make a the bigger one and b the lesser one
	swapped = False
	if a < b:
		a, b = b, a
		swapped = True
	# ca and cb store current a and b in form of
	# coefficients with initial a and b
	# a' = ca[0] * a + ca[1] * b
	# b' = cb[0] * a + cb[1] * b
	ca = (1, 0)
	cb = (0, 1)
	while b != 0:
		# k denotes how many times number b
		# can be substracted from a
		k = a // b
		# swap a and b so b is always the lesser one
		a, b, ca, cb = b, a-b*k, cb, (ca[0]-k*cb[0], ca[1]-k*cb[1])
	if swapped:
		return (ca[1], ca[0])
	else:
		return ca

In [3]:
#Function to calculate MOD POWER
#Iterating from 1 to e and multiplying by res by b, it iterates from MSB of e to LSB and at each iteration it does r=r*r+bit. 
# At any step if r=b^x then after appending it becomes r=b^(x^2+bit)
def modpow(b, e, n):
	# find length of e in bits
	tst = 1
	siz = 0
	while e >= tst:
		tst <<= 1
		siz += 1
	siz -= 1
	# calculate the result
	r = 1
	for i in range(siz, -1, -1):
		#Updating the required r
		r = (r * r) % n
		#If the bit at ith place is 1 then we are multplying by b
		if (e >> i) & 1: r = (r * b) % n
	return r

In [4]:
#Defining Matrix Multiplication 
def sqmatrixmul(m1, m2, w, mod):
	mr = [[0 for j in range(w)] for i in range(w)]
	for i in range(w):
		for j in range(w):
			for k in range(w):
				mr[i][j] = (mr[i][j] + m1[i][k] * m2[k][j]) % mod
	return mr


In [5]:
# fibonacci calculator
#Function to calculate the xth fibonacci number
def fib(x, mod):
	if x < 3: return 1
	x -= 2
	# find length of e in bits
	tst = 1
	siz = 0
	while x >= tst:
		tst <<= 1
		siz += 1
	siz -= 1
	# calculate the matrix
	fm = [
		# function matrix
		[0, 1],
		[1, 1]
	]
	rm = [
		# result matrix
		# (identity)
		[1, 0],
		[0, 1]
	]
	for i in range(siz, -1, -1):
		rm = sqmatrixmul(rm, rm, 2, mod)
		if (x >> i) & 1:
			rm = sqmatrixmul(rm, fm, 2, mod)

	# second row of resulting vector is result
	return (rm[1][0] + rm[1][1]) % mod

In [6]:
#FUNCTION TO GENERATE PRIME NUMBER OF GIVEN SIZE 
def genprime(siz):
	while True:
		#Generating a random number of the order
		num = (1 << (siz - 1)) + secrets.randbits(siz - 1) - 10;
		# num must be 3 or 7 (mod 10)
		num -= num % 10
		num += 3 # 3 (mod 10)
		# heuristic test
		if modpow(2, num - 1, num) == 1 and fib(num + 1, num) == 0:
			return num
		num += 5 # 7 (mod 10)
		# heuristic test
		if modpow(2, num - 1, num) == 1 and fib(num + 1, num) == 0:
			return num

In [7]:
p=genprime(1024)
print(p)

108127343276583416498856174169751649126480346282185417931813004656473734560500587128340692884702827501973058366284853288410112152386186261547171454734409116069544799336770519128441095917851559232084119372403856316429049393710563408247555992271115578201412430718003036524328109388045864731799647787735008273983


In [7]:
#FUNCTION TO GENERATE KEY
def create_keys(size):
	#Generating the primes
	p = genprime(size//2)
	q = genprime(size//2)

	#Calculating the Modulo 
	n = p * q
	
	#Calculating the Tollen Number for it to calculate the private exponent
	lambda_n = (p - 1) * (q - 1)
	e = 35537
	d = eucalg(e, lambda_n)[0]
	if d < 0: d += lambda_n #If d is negative adding lambda_n
	return (d, n),(e, n)

#FUNCTION TO ENCRYPT DATA
def encrypt_bytes(data, key):
	data=str(data)#Converting Data to string to 
	data=data.encode('latin-1')
	#data=udata.decode('latin-1')
	#data.encode()
	#print(data)
	##print(sizeof(data))
	#data=data.encode(encoding="latin-1",errors:'ignore')
	#data.encode('Latin-1')
	data = bytearray(data)
	#print(data)
	#print(len(data))
	cdata = bytearray()
	for i in range(0, len(data), 256):
		#read 256 bytes and store as long
		# to m
		m = 0
		for j in range(256):
			if i + j < len(data):
				m = (m << 8) + data[i + j]
			else:
				m <<= 8
		# encrypt m
		k = modpow(m, key[0], key[1])
		# store c into cdata
		for j in range(255, -1, -1):
			cdata.append((k >> (j * 8)) & 255)
		#print(len(cdata))
		#print(cdata)	
	#cdata=cdata.decode()
	#cdata.encode(encoding='UTF-8',errors='ignore')
	#cdata=bytes(cdata)
	cdata=cdata.decode('latin-1')
	#print(type(cdata))
	#print(cdata)
	return cdata

In [116]:
def encrypt_bytes(data, key):
	#data=str(data)
	data = bytearray(str(data),encoding='latin-1')
	cdata = bytearray()
	for i in range(0, len(data), 256):
		# read 256 bytes and store as long
		# to m
		m = 0
		for j in range(256):
			if i + j < len(data):
				m = (m << 8) + data[i + j]
			else:
				m <<= 8
		# encrypt m
		c = modpow(m, key[0], key[1])
		# store c into cdata
		for j in range(255, -1, -1):
			cdata.append((c >> (j * 8)) & 255)
	return bytes(cdata)

# both functions are essencially the same,
# the only difference is in which key you use
decrypt_bytes = encrypt_bytes

def decrypt_bytes(data,key):
    decrypted=encrypt_bytes(data,key)
    #decrypted=str(decrypted,'latin-1')
    #decrypted=decrypted.encode('latin-1')
    #decrypted=decrypted.decode('utf-8',errors='ignore')
    #print(data)
    #print(type(decrypted))
    
    return decrypted

In [10]:
public,private=create_keys(1024)

In [117]:
data="234"
data=encrypt_bytes(data,public)
data=decrypt_bytes(data,private)
data=data.decode('latin-1')
#data=str(data)
print(data)

                                                                                                                                (öÃtÓ¤ðLa*ó~ù«DT|H+ji9:ÃF?WhXÇ0¹Æå³¨Ià»çÌ*¾&m\1fûùIÌù}²;UQd%ÄD0$:N´¶"Í9º¯d^hìU|y
&0¶zgÞh³­õxC;©·R`Õ^÷è}Ð¤¡é q«êc¼.¯áô'âÔÍ}Õ                                                                                                                                Bkã« ¨·âï®§SðûÝ}Ï¬ö Ìí<­ðÞÍ/ÊKË·¬eÓèe¬Ðótë3¤cÎ[1ôÄZµÎw 6¹WÙÃBs´'ë.-A¹×>9an¿RÅÐÄÙÊ´riçÉöÆujÑ¬                                                                                                                                5£»üÊç.UjåMÒÖÅüÉn¼u1{ì1G¯­^ðü_C¼0ÎÞôw37M¦E¦mÛ÷	ãâ¿1r#±¯Hª#3[³zÖE(òìÄ'õ1 òüêN}2¶÷nïìhýçéu³y´Jn6É


WE ARE IMPLEMENTING THE WIENER'S ATTACK ON SHORT EXPONENT D USING METHOD OF CONTINUED FRACTIONS 

In [98]:
##Function to Calculate Continued Fractional of a fraction of form (n/d)
def cf_exp(num, den):
    #print(type(den))
    exp=[]
    q=num//den
    r=num%den
    exp.append(q)
    while  r!=0:
        num,den=den,r
        q=num//den
        r=num%den
        exp.append(q)
    return exp    

In [99]:
#Function to calculate the convergent of a continued fraction n
def convergent(n):
    num=[] #Numator
    deno=[] #Denominator
    for i in range(len(n)):
        if i==0:
            num_i=n[i]
            deno_i=1
        elif i==1:
            num_i= n[i]*n[i-1]+1
            deno_i=n[i]
        else:
            num_i=n[i]*num[i-1]+num[i-2]
            deno_i=n[i]*deno[i-1]+deno[i-2]
        num.append(num_i)
        deno.append(deno_i)
        yield(num_i,deno_i)
    #return (num,deno)              


In [100]:
##Importing the required Libraries Hashlib for hashing
import gmpy2,sys
from sympy import *
from gmpy2 import c_div, isqrt

In [101]:
##Function to test Key-pair
def test_key(n,e,d):
    msg=(n-123)>>7
    c=modpow(msg,e,n)
    return modpow(c,d,n)==msg

In [102]:
##Function to generate the Wiener key-pair satisfying the criteria on private exponent
def key_pair_wiener(size):
    while True:
        p=genprime(size//2)
        q=genprime(size//2)
        if q<p<2*q:
            break
    n=p*q
    phi_n=(p-1)*(q-1)

    #Calculating Maximum D and bits in it 
    max_d=c_div((isqrt(isqrt(n))),3)
    max_d_bits=max_d.bit_length()-1

    while True:
        #Generating random d of specified bits
        d=secrets.randbits(max_d_bits)
        #Calculating inverse Modulo for d i.e e or public exponent 
        try:
            e=int(gmpy2.invert(d, phi_n))
        except ZeroDivisionError:
            continue
        if (e*d)%phi_n==1:
            break

    #Testing if the key pair is vaild     
    assert test_key(n,e,d)
    
    #Returning the values
    return n, e, d, p, q     
        

In [103]:
if __name__ == '__main__':
    #Generating a pair of private and public key
    #Hashing it to make it more secure
    N, e, d, p, q = key_pair_wiener(1024)
    print('[+] Generated an RSA keypair with a short private exponent.')
    print('[+] ++ e:    ', (e))
    print('[+] -- d:    ', (d))
    print('[+] ++ N:    ', (N))
    print('[+] -- p:    ', (p))
    print('[+] -- q:    ', (q))
    print('[+] -- phiN: ', ((p - 1)*(q - 1)))
    print('[+] ------------------')

    #Generating the cf_expansion of e and N
    cf_expansion = cf_exp(e, N)
    
    #Calculating the convergence of the expansion
    convergents = convergent(cf_expansion)
    print('[+] Found the continued fractions expansion convergents of e/N.')

    print('[+] Iterating over convergents; '
            'Testing correctness through factorization.')
    print('[+] ...')
    for pk, pd in convergents: # pk - possible k, pd - possible d
        if pk == 0:
            continue;

        possible_phi = (e*pd - 1)//pk

        p = Symbol('p', integer=True)
        roots = solve(p**2 + (possible_phi - N - 1)*p + N, p)

        if len(roots) == 2:
            pp, pq = roots # pp - possible p, pq - possible q
            if pp*pq == N:
                print('[+] Factored N! :) derived keypair components:')
                print('[+] ++ e:    ', (e))
                print('[+] ++ d:    ', (pd))
                print('[+] ++ N:    ', (N))
                print('[+] ++ p:    ', (pp))
                print('[+] ++ q:    ', (pq))
                print('[+] ++ phiN: ', (possible_phi))
                sys.exit(0)

    print('[-] Wiener\'s Attack failed; Could not factor N')
    sys.exit(1)

[+] Generated an RSA keypair with a short private exponent.
[+] ++ e:     60308644142284766819162858097749715800105225991403017523608095259298997457296703342430885290037156178001435087839315286122213968619770087667481401815715377974134072830100453119320018542821793197390255507565115387655761627631343010784450904796610081438599648693207190750180704977997792619297922268035727142427
[+] -- d:     5331867266387788009502181936198395258153087173479822263864248828278569914835
[+] ++ N:     117964626161945173834889393077732771831318487010014367911702452686073141273643919085806218092030199959726710555363311557087332575829695506225305180649252954377219252121837762893343859064149434729693013637458559798828179393610911699298030627881508683518315722378625447470591881053271596051644295533668023758559
[+] -- p:     13039305930738824641465810590384310144646667024777703840012501310145426526849257631750952543147825154994507724136737013995513801144204796693556525503368593
[+] -- q:     904684856606176270

SystemExit: 0

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3449: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


Håstad Broadcast Attack on RSA 

In [94]:
##importing Required Libraries
from math import ceil, gcd
from mpmath import mp

In [93]:
##The intercepted cipher text and the Corresponding Moduli
C1 = 0x94f145679ee247b023b09f917beea7e38707452c5f4dc443bba4d089a18ec42de6e32806cc967e09a28ea6fd2e683d5bb7258bce9e6f972d6a30d7e5acbfba0a85610261fb3e0aac33a9e833234a11895402bc828da3c74ea2979eb833cd644b8ab9e3b1e46515f47a49ee602c608812241e56b94bcf76cfbb13532d9f4ff8ba
N1 = 0xa5d1c341e4837bf7f2317024f4436fb25a450ddabd7293a0897ebecc24e443efc47672a6ece7f9cac05661182f3abbb0272444ce650a819b477fd72bf01210d7e1fbb7eb526ce77372f1aa6c9ce570066deee1ea95ddd22533cbc68b3ba20ec737b002dfc6f33dcb19e6f9b312caa59c81bb80cda1facf16536cb3c184abd1d5
C2 = 0x5ad248df283350558ba4dc22e5ec8325364b3e0b530b143f59e40c9c2e505217c3b60a0fae366845383adb3efe37da1b9ae37851811c4006599d3c1c852edd4d66e4984d114f4ea89d8b2aef45cc531cfa1ab16c7a2e04d8884a071fed79a8d30af66edf1bbbf695ff8670b9fccf83860a06e017d67b1788b19b72d597d7d8d8
N2 = 0xaf4ed50f72b0b1eec2cde78275bcb8ff59deeeb5103ccbe5aaef18b4ddc5d353fc6dc990d8b94b3d0c1750030e48a61edd4e31122a670e5e942ae224ecd7b5af7c13b6b3ff8bcc41591cbf2d8223d32eeb46ba0d7e6d9ab52a728be56cd284842337db037e1a1da246ed1da0fd9bdb423bbe302e813f3c9b3f9414b25e28bda5
C3 = 0x8a9315ee3438a879f8af97f45df528de7a43cd9cf4b9516f5a9104e5f1c7c2cdbf754b1fa0702b3af7cecfd69a425f0676c8c1f750f32b736c6498cac207aa9d844c50e654ceaced2e0175e9cfcc2b9f975e3183437db73111a4a139d48cc6ce4c6fac4bf93b98787ed8a476a9eb4db4fd190c3d8bf4d5c4f66102c6dd36b73
N3 = 0x5ca9a30effc85f47f5889d74fd35e16705c5d1a767004fec7fdf429a205f01fd7ad876c0128ddc52caebaa0842a89996379ac286bc96ebbb71a0f8c3db212a18839f7877ebd76c3c7d8e86bf6ddb17c9c93a28defb8c58983e11304d483fd7caa19b4b261fc40a19380abae30f8d274481a432c8de488d0ea7b680ad6cf7776b

In [95]:
##Checking if any of three are co-prime, else we can easily factor the common term to find the primes
assert gcd(N1, N2) == gcd(N1, N3) == gcd(N2, N3) == 1

In [96]:
# Using Chinese Remainder Theorem
# Z_p x Z_q isomorph with Z_N where N=pq (where Z_p is a Ring of integers redcued by p)
# (x_p, x_q) <-> [(x_p * 1_p + x_q * 1_q) mod N](where 1_p is the identity element in ring p and 1_q is the identity element in q)
# 1_p <-> (1, 0)
# 1_q <-> (0, 1)
N12 = N1 * N2
#Using the extended Eculiedan 
k= eucalg(N1, N2)
a,b=k[0],k[1]
p1 = b * N2 % N12
q1 = a * N1 % N12
C12 = (C1 * p1 + C2 * q1) % N12

N = N12 * N3

k = eucalg(N12, N3)
a,b=k[0],k[1]
p1 = b * N3 % N
q1 = a * N12 % N
C123 = (C12 * p1 + C3 * q1) % N

mp.dps = len(str(C123))     # set floating-point precision of mpmath

m = int(mp.cbrt(C123))
bits = bin(m)[2:]
message = int.to_bytes(m, ceil(len(bits) / 8), byteorder="big").decode("ASCII")
print(message)

This message is secret and unfortunately e is not high. The message should also be sufficiently long so it is a residue.
